In [2]:
!pip install transformers

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
import string
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
warnings.filterwarnings('ignore')



In [4]:
train=pd.read_csv("./train.csv")

In [5]:
test=pd.read_csv('./test.csv')
submission=pd.read_csv('sample_submission.csv')

In [6]:
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
Distillbert_model, DistillBert_tokenizer, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [8]:
tokenizer=DistillBert_tokenizer.from_pretrained(pretrained_weights)

In [9]:
model = Distillbert_model.from_pretrained(pretrained_weights)

In [10]:
train['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [10]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

train['text']=train['text'].apply(lambda x : remove_URL(x))
test['text']=test['text'].apply(lambda x : remove_URL(x))

In [11]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

train['text']=train['text'].apply(lambda x : remove_html(x))
test['text']=test['text'].apply(lambda x : remove_html(x))

In [12]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

train['text']=train['text'].apply(lambda x : remove_emoji(x))
test['text']=test['text'].apply(lambda x : remove_emoji(x))

In [15]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

train['text']=train['text'].apply(lambda x : remove_punct(x))
test['text']=test['text'].apply(lambda x : remove_punct(x))

In [11]:
tokenized_train_tweet=train['text'].apply((lambda x : tokenizer.encode(x, add_special_tokens=True)))

In [12]:
test.head(3)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."


In [13]:
tokenized_test_tweet=test['text'].apply((lambda x : tokenizer.encode(x, add_special_tokens=True)))

In [14]:
max_len = 0
for i in tokenized_train_tweet.values:
    if len(i) > max_len:
        max_len = len(i)

In [15]:
print(max_len)

84


In [16]:
padded_train_tweet=np.array([i + [0]*(max_len-len(i)) for i in tokenized_train_tweet.values])

In [17]:
padded_test_tweet=np.array([i + [0]*(max_len-len(i)) for i in tokenized_test_tweet.values])

In [18]:
padded_train_tweet.shape

(7613, 84)

In [19]:
padded_test_tweet.shape

(3263, 84)

In [20]:
attention_mask_train=np.where(padded_train_tweet != 0,1,0)

In [21]:
attention_mask_train.shape


(7613, 84)

In [22]:
attention_mask_test=np.where(padded_test_tweet != 0,1,0)

In [23]:
attention_mask_test.shape[0]

3263

In [24]:
input_ids_train = torch.tensor(padded_train_tweet)  
attention_mask_train = torch.tensor(attention_mask_train)


In [25]:
input_ids_test=torch.tensor(padded_test_tweet)
attention_mask_test=torch.tensor(attention_mask_test)

In [26]:
train_data=TensorDataset(input_ids_train, attention_mask_train)
batch_size=23
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [27]:
test_data=TensorDataset(input_ids_test, attention_mask_test)
batch_size_test=13
test_sampler=SequentialSampler(test_data)
test_dataloader=DataLoader(test_data, sampler=test_sampler, batch_size=batch_size_test)

In [28]:
features=np.zeros((331, batch_size, 768))

In [29]:
features_test=np.zeros((251, batch_size_test, 768))

In [30]:
print(features.shape)

(331, 23, 768)


In [1]:
i=0
for batch in train_dataloader:
    with torch.no_grad():
        input_ids=batch[0]
        input_mask=batch[1]
        last_hidden_states = model(input_ids, attention_mask=input_mask)
        features[i, :, :]=last_hidden_states[0][:,0,:].numpy()
        i=i+1

NameError: name 'train_dataloader' is not defined

In [32]:
features1=features.reshape(331*batch_size, 768)

In [33]:
features1.shape

(7613, 768)

In [34]:
features[1, 0, :]

array([-5.39203659e-02,  3.03787235e-02,  1.39179111e-01, -1.76852122e-01,
       -7.24829733e-02, -2.75498092e-01,  2.07533076e-01,  6.46994054e-01,
       -1.89658925e-01, -9.60106999e-02, -1.15032047e-02, -1.14084512e-01,
       -5.30006588e-02,  5.75991511e-01,  1.03690274e-01,  5.88982627e-02,
       -1.17204510e-01,  2.89246082e-01,  7.78878182e-02, -1.90412909e-01,
       -1.76591566e-03, -3.35517049e-01, -1.64437648e-02,  7.18899146e-02,
        1.29146010e-01, -2.23933533e-01, -7.83770978e-02, -6.89592510e-02,
        2.46284306e-01, -1.59113090e-02, -9.64733735e-02,  3.07346638e-02,
        4.66570482e-02, -9.82529111e-03, -6.35418221e-02,  6.92062527e-02,
       -1.07133418e-01, -1.84001252e-02, -1.33319885e-01,  6.36131540e-02,
       -9.52297822e-02, -1.39796570e-01,  2.56442696e-01, -1.09595932e-01,
       -9.95302722e-02, -1.36787862e-01, -2.20107603e+00, -1.15006324e-02,
        4.46680710e-02, -1.79410204e-01,  2.66246557e-01,  1.64560884e-01,
        1.64374426e-01,  

In [40]:
i=0
for batch in test_dataloader:
    with torch.no_grad():
        input_ids=batch[0]
        input_mask=batch[1]
        last_hidden_states = model(input_ids, attention_mask=input_mask)
        features_test[i, :, :]=last_hidden_states[0][:,0,:].numpy()
        i=i+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [41]:
feature_test=features_test.reshape(251*batch_size_test, 768)

In [43]:
np.save('fetures_distill.npy', features1)

In [44]:
np.save('features_test.npy', feature_test)